In [1]:
import os
import json
import yaml

In [2]:
import sys 
import torch
sys.path.append('..')

import os
import pytorch_lightning as pl
from src.datasets.vqa_rad_dataset import VQRadDataModule
from src.models.multimodal.flamingo_module import FlamingoModule

from pytorch_lightning import Trainer, seed_everything
import torchvision.transforms as T

from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning import loggers as pl_loggers

from src.utils.utils import load_config

In [3]:
if __name__ == '__main__':
    seed_everything(42, workers=True)

    args = load_config('/u/home/koksal/mlmi-vqa/configs','config.yaml')
     
    #img_mean = (0.48,0.48,0.48)
    #img_std = (0.265,0.265,0.265)
    #img_mean = (0.48,0.48,0.48)
    #img_std = (0.265,0.265,0.265)

    augmentations = {'train':
        T.Compose(
        [   
            T.Resize((args['train']['augmentation']['resize_size'])),
            T.RandomRotation((args['train']['augmentation']['random_rotation'])),
            T.ToTensor(),
            T.Normalize(mean=args['dataset']['mean'], std=args['dataset']['std'])
        ]),
        'val':
        T.Compose(
        [
            T.Resize((args['test']['augmentation']['resize_size'])),
            T.RandomRotation((args['test']['augmentation']['random_rotation'])),
            T.ToTensor(),
            T.Normalize(mean=args['dataset']['mean'], std=args['dataset']['std'])
        ]),
        'test':
        T.Compose(
        [
            T.Resize((args['test']['augmentation']['resize_size'])),
            T.ToTensor(),
            T.Normalize(mean=args['dataset']['mean'], std=args['dataset']['std'])
        ])
    }


    args = load_config('/u/home/koksal/mlmi-vqa/configs','config.yaml')


    mimic_datamodule = VQRadDataModule(args, augmentations= augmentations)


    train_loader = mimic_datamodule.train_dataloader()
    val_loader = mimic_datamodule.val_dataloader()





Global seed set to 42


Load memory False
Samples length  train :  3064
answers length  train :  3064
Samples length  test :  451
answers length  test :  451
Training split length  3064
Test split length  451
Test split length after removal  408
Num unique answers vqa-rad  458
Num unique answers vqa-rad  458
Num unique answers vqa-rad  458


In [4]:
mimic_datamodule.label2ans

['yes',
 'no',
 'ultrasound',
 'extremities',
 'left temporal lobe',
 'cardiovascular',
 'vasculature',
 'mri diffusion weighted',
 'brain',
 'chest xray',
 'axial',
 'asymmetric',
 'pulmonary nodules',
 'free air',
 'hypodense lesion',
 'gi',
 'ring enhancing lesion',
 'diffuse',
 'ct',
 'pancreas',
 'right upper lobe',
 'hypodense',
 'left hepatic lobe',
 'lungs',
 'bilateral',
 'cirrhosis',
 'atherosclerotic calcification',
 'right parietal lobe',
 'chest',
 'metastasis',
 'enlarged',
 'volume loss',
 '1',
 'extraluminal air and small fluid collection',
 'nodular opacities',
 'extra axial and at right choroidal fissure',
 'prior surgery',
 'ring enhancing lesions',
 'right temporal lobe',
 'gallstones',
 'right',
 'abscess',
 'cystic lesions',
 'middle mogul',
 'right lung',
 'enlarged fluid filled',
 'pancreatic body',
 'right lobe of liver',
 'right hemisphere',
 'right pca',
 'basal ganglia',
 'right superior cavoatrial junction',
 'ischemia',
 'suprasellar cistern',
 'above',
 '

In [5]:
mimic_datamodule.train_dataset.tokenizer 

PreTrainedTokenizerFast(name_or_path='emilyalsentzer/Bio_ClinicalBERT', vocab_size=28996, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['<image>', '<EOC>', '<EOQ>']})

In [ ]:
unique_val_dict = {}
for question in mimic_datamodule.val_split:
    unique_val_dict[question['qid']] = 1

In [ ]:
unique_train_dict = {}
for question in mimic_datamodule.train_split:
    unique_train_dict[question['qid']] = 1

In [ ]:
len(set(unique_train_dict.keys()))

In [ ]:
len(set(unique_val_dict.keys()))

In [ ]:
set.intersection(set(unique_train_dict.keys()), set(unique_val_dict.keys()))

In [ ]:
cnt = 0
for each in mimic_datamodule.val_split:
    if each['answer']['labels'] == None:
        cnt +=1


In [ ]:
mimic_datamodule.val_split[122]

In [ ]:
cnt

In [ ]:
sample1 = mimic_datamodule.train_dataset[1111]
sample2= mimic_datamodule.train_dataset[1465]

In [ ]:
for k in sample1.keys():
    print('K : ',k , 'values are : ', sample1[k],'           '  ,sample2[k] )

In [ ]:
mimic_datamodule.train_dataset[1465].keys()

In [6]:
model = FlamingoModule(args) 

Clip architecture is being loaded
Clip pretrained weights are being loaded
Flamingo is being initialized with  bert-clinical  as language model


You are using a model of type bert to instantiate a model of type bert-generation. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertGenerationDecoder: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'bert.pooler.dense.bias', 'bert.embeddings.token_type_embeddings.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertGenerationDecoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertGenerationDecoder from the ch

Perceiver Resampler is being initialized
Perceiver attention is created with  dim :  768 dim_head:  64
Perceiver attention is created with  dim :  768 dim_head:  64
Perceiver Resampler is initialized
img encoder outdim initialized
Layers are initializzed
Flamingo is initalized
Self classifier  Sequential(
  (0): LayerNorm()
  (1): Dropout(p=0.1, inplace=False)
  (2): Linear(in_features=1536, out_features=458, bias=True)
)


In [7]:
mimic_datamodule.train_dataset[12]

{'question': 'is there a pneumothorax',
 'label': 1,
 'index_eoq': 12,
 'input_ids': tensor([[  101, 28996,  2304,   131,  1110,  1175,   170,   185,  1673,  1818,
          12858, 25632, 28998,   102,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'targets': tensor([[28996,  2304,   131,  1110,  1175,   170,   185,  1673,  1818, 12858,
          25632, 28998,   102,     0,     0,     0,     0,   

In [8]:
dataloader_val = mimic_datamodule.val_dataloader()
dataloader_train = mimic_datamodule.train_dataloader()

In [9]:
for batch in dataloader_val:
    print(batch.keys())
    break



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [10]:
batch.keys()

dict_keys(['question', 'label', 'index_eoq', 'input_ids', 'token_type_ids', 'targets', 'ID', 'qa_pair', 'image'])

In [11]:
batch['input_ids'].isnan().any()

tensor(False)

In [12]:
batch.keys()

dict_keys(['question', 'label', 'index_eoq', 'input_ids', 'token_type_ids', 'targets', 'ID', 'qa_pair', 'image'])

In [13]:
batch['qa_pair']

['<image> question: is there evidence of an aortic aneurysm <EOQ>',
 '<image> question: is there airspace consolidation on the left side <EOQ>',
 '<image> question: is this an axial plane <EOQ>',
 '<image> question: is there vascular pathology in this image <EOQ>',
 '<image> question: is there cardiomyopathy <EOQ>',
 '<image> question: are >12 ribs present in the image <EOQ>',
 '<image> question: does this represent adequate inspiratory effort <EOQ>',
 '<image> question: is there a rib fracture <EOQ>',
 '<image> question: is there evidence of any fractures of the ribs <EOQ>',
 '<image> question: is there any intraparenchymal abnormalities in the lung fields <EOQ>',
 '<image> question: is there evidence of any abnormalities of the lung parenchyma <EOQ>',
 '<image> question: is there evidence of small bowel obstruction on this image <EOQ>',
 '<image> question: is there small bowel thickening present <EOQ>',
 '<image> question: was the patient positioned appropriately \twithout tilting <E

In [14]:
mimic_datamodule.train_dataset.tokenizer.all_special_ids[mimic_datamodule.train_dataset.tokenizer.all_special_tokens.index('<image>')]

28996

In [15]:
eoc_token_id = mimic_datamodule.train_dataset.tokenizer.all_special_ids[mimic_datamodule.train_dataset.tokenizer.all_special_tokens.index('<image>')]
eoc_token_id

28996

In [16]:
mimic_datamodule.train_dataset.tokenizer

PreTrainedTokenizerFast(name_or_path='emilyalsentzer/Bio_ClinicalBERT', vocab_size=28996, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['<image>', '<EOC>', '<EOQ>']})

In [17]:
mimic_datamodule.train_dataset.tokenizer.pad_token_id

0

In [18]:
val_data_iterator = iter(dataloader_val)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [19]:
batch = next(val_data_iterator)

In [20]:
batch.keys()

dict_keys(['question', 'label', 'index_eoq', 'input_ids', 'token_type_ids', 'targets', 'ID', 'qa_pair', 'image'])

In [32]:
batch['input_ids']

tensor([[[  101, 28996,  2304,  ...,     0,     0,     0]],

        [[  101, 28996,  2304,  ...,     0,     0,     0]],

        [[  101, 28996,  2304,  ...,     0,     0,     0]],

        ...,

        [[  101, 28996,  2304,  ...,     0,     0,     0]],

        [[  101, 28996,  2304,  ...,     0,     0,     0]],

        [[  101, 28996,  2304,  ...,     0,     0,     0]]])

In [35]:
batch["token_type_ids"]

tensor([[[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]]])

: 

In [22]:
images = batch["image"]
input_tokens = batch["input_ids"]
token_type_ids = batch["token_type_ids"] 
targets = batch["targets"]
batch_size = images.shape[0]

class_labels = batch["label"]
index_eoq = batch["index_eoq"]
flamingo_logits, token_embeds, image_embeddings = model.flamingo_palm(
        input_tokens.squeeze(1), images.unsqueeze(1), token_type_ids.unsqueeze(1),
        return_image_embeddings = True)


In [31]:
input_tokens = batch["input_ids"]

In [29]:
for batch in train_loader:
    images = batch["image"]
    input_tokens = batch["input_ids"]
    if batch["input_ids"].isnan().any() ==True:
        print('found nan ')
        break
    token_type_ids = batch["token_type_ids"] 
    targets = batch["targets"]
    batch_size = images.shape[0]

    class_labels = batch["label"]
    index_eoq = batch["index_eoq"]
    flamingo_logits, token_embeds, image_embeddings = model.flamingo_palm(
            input_tokens.squeeze(1), images.unsqueeze(1), token_type_ids.unsqueeze(1),
            return_image_embeddings = True)



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [ ]:
batch['index_eoq']

In [ ]:
token_embeds.shape

In [ ]:
def aggregate_tokens_until_eoc(token_embeds, eoq_indices, agg_func='mean'):
    aggregated_tensors = []
    for i, row in enumerate(token_embeds):
        if agg_func == 'mean':
            aggregated_tensors.append(torch.mean(row[:batch['index_eoq'][i],:],dim=0))
    return torch.stack(aggregated_tensors)

In [ ]:
aggregate_tokens_until_eoc(token_embeds, batch['index_eoq']).shape

In [ ]:
source = torch.tensor([[1,2,3], [4,5,6], [7,8,9], [10,11,12]])
source, source.shape

In [ ]:
index = torch.tensor([[0,0],[1,1],[2,2],[0,1]])
index

In [ ]:
source.gather(dim=1, index=index)

In [ ]:
source.gather(dim=1, index=index)

In [ ]:
torch.concat(torch)

In [ ]:
eoq_embeds = token_embeds[torch.arange(batch_size), index_eoq]

In [ ]:
eoq_embeds.shape

In [ ]:
eoq_embeds.shape

In [ ]:
with open(os.path.join(args['dataset']['vqa_rad_path'],'VQA-RAD_public.json'), 'r') as f:
        sample_dicts = json.load(f)

In [ ]:
len(sample_dicts)

In [ ]:
test_set = []
train_set = []
for sample in sample_dicts:
    if sample['phrase_type'].startswith('test'):
        test_set.append(sample)
    else:
        train_set.append(sample)


In [ ]:
print('len test set', len(test_set), 'len train set', len(train_set))

In [ ]:
test_set

In [ ]:
test_set[0]

In [ ]:
# Create answers dictionary
answer_list_concat= []
for sample in sample_dicts:
    answer_list_concat.append(str(sample['answer']).strip().lower())

# Create a dictionary of answers
answer_to_label = {}
label_to_answer = {}
for i,ans in enumerate(set(answer_list_concat)):
    answer_to_label[ans] = i
    label_to_answer[i] = ans

In [ ]:
# Create answers dictionary
answer_list_train = []
for sample in train_set:
    answer_list_train.append(str(sample['answer']).strip().lower())

# Create a dictionary of answers
answer_to_label_train = {}
label_to_answer_train = {}
for i,ans in enumerate(set(answer_list_train)):
    answer_to_label_train[ans] = i
    label_to_answer_train[i] = ans

In [ ]:
# Create answers dictionary
answer_list_test = []
for sample in test_set:
    answer_list_test.append(str(sample['answer']).strip().lower())

# Create a dictionary of answers
answer_to_label_test = {}
label_to_answer_test = {}
for i,ans in enumerate(set(answer_list_test)):
    answer_to_label_test[ans] = i
    label_to_answer_test[i] = ans

In [ ]:
import seaborn
from collections import Counter
import pandas as pd
import plotly.express as px

In [ ]:
test_counter = Counter(answer_list_test)

In [ ]:
most_common_dict = {k:v for (k,v) in test_counter.most_common(10)}

In [ ]:
df = pd.DataFrame.from_dict(most_common_dict,orient='index', columns=['count'])

In [ ]:
df

In [ ]:
len(set(answer_list_train))

In [ ]:
len(set(answer_list_test))

In [ ]:
count = 0
not_included_in_train = []
for test_answer in answer_list_test:
    if test_answer in set(answer_list_train):
        count +=  1
    else:
        not_included_in_train.append(test_answer)


In [ ]:
loaded_train_answers = json.load(open('/u/home/koksal/mlmi-vqa/data/answer2label_vqarad.json'))

In [ ]:
test_set

In [ ]:
cnt = 0
not_included_in_train = []
for sample in test_set:
    if sample['answer'].lower().strip() in loaded_train_answers.keys():
        cnt += 1
    else:
        not_included_in_train.append(sample)

In [ ]:
cnt

In [ ]:
len(answer_list_test)

In [ ]:
not_included_in_train

In [ ]:
not_included_in_train

In [ ]:
count

In [ ]:
px.bar(df,title='VQARad Test Answer Counts',labels={'value':'count','index':'x_legend','variable':'legend'})

In [ ]:
answer_to_label_test

In [ ]:
answer_set_test = set(label_to_answer_test.values()) 

In [ ]:
answer_set_train = set(label_to_answer.values())

In [ ]:
len(answer_set_train)

In [ ]:
len(answer_set_test)

In [ ]:
only_test_answers = answer_set_test.difference(answer_set_train)

In [ ]:
only_test_answers

In [ ]:
train_included_tst_answer_cnt = 0
for qa in test_set:
    if qa['answer'].strip().lower() in answer_set_train:
        train_included_tst_answer_cnt += 1


In [ ]:
len(test_set)

In [ ]:
train_included_tst_answer_cnt

In [ ]:
336/451

In [ ]:
only_test_answers

In [ ]:
'a bit' in answer_set_train

In [ ]:
answer_set_train

In [ ]:
print('Num unique answers vqa-rad ', len(answer_to_label.keys()))

In [ ]:
answer_to_label

In [ ]:
import json
with open('/u/home/koksal/mlmi-vqa/data/answer2label_vqarad_concat.json', 'w') as f:
    json.dump(answer_to_label, f)

In [ ]:
import json
with open('/u/home/koksal/mlmi-vqa/data/label2answer_vqarad_concat.json', 'w') as f:
    json.dump(label_to_answer, f)

In [ ]:
print('Lenght of Training Set ',len(train_set), 'length test set', len(test_set))

In [ ]:
sample_dicts[0]

In [ ]:
import sys 
import torch
sys.path.append('..')

import os
import pytorch_lightning as pl
from src.datasets.vqa_rad_dataset import VQRadDataModule
from src.models.multimodal.flamingo_module import FlamingoModule

from pytorch_lightning import Trainer, seed_everything
import torchvision.transforms as T

from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning import loggers as pl_loggers

from src.utils.utils import load_config

if __name__ == '__main__':
    seed_everything(42, workers=True)

    img_mean = (0.48,0.48,0.48)
    img_std = (0.265,0.265,0.265)

    augmentations = {'train':
        T.Compose(
        [   
            T.Resize((224,224)),
            T.RandomRotation(10),
            T.ToTensor(),
            T.Normalize(mean=img_mean, std=img_std)
        ]),
        'val':
        T.Compose(
        [
            T.Resize((224,224)),
            T.RandomRotation(10),
            T.ToTensor(),
            T.Normalize(mean=img_mean, std=img_std)
        ]),
        'test':
        T.Compose(
        [
            T.Resize((224,224)),
            T.ToTensor(),
            T.Normalize(mean=img_mean, std=img_std)
        ])
    }


    args = load_config('/u/home/koksal/mlmi-vqa/configs','config.yaml')

    args['train']['batch_size']=1
    datamodule = VQRadDataModule(args, augmentations= augmentations)

    train_loader = datamodule.train_dataloader()
    val_loader = datamodule.val_dataloader()



    model = FlamingoModule(args) 

    if args['pretrained']:
        print("Pretrained Flamingo Model is loaded from checkpoint : ",args['pretrained'])
        model.load_state_dict(torch.load(args['pretrained'])["state_dict"])


    from pytorch_lightning.callbacks import ModelCheckpoint
    from pytorch_lightning.callbacks.early_stopping import EarlyStopping


In [ ]:
args['pretrained'] = '/u/home/koksal/mlmi-vqa/notebooks/lightning_logs/version_3/checkpoints/epoch=2-val_acc_epoch=0.00-val_total_loss_epoch=11.29-val_loss_generation_epoch=4.15-val_classification_loss_epoch=7.14.ckpt'

In [ ]:
if args['pretrained']:
    print("Pretrained Flamingo Model is loaded from checkpoint : ",args['pretrained'])
    model.load_state_dict(torch.load(args['pretrained'])["state_dict"])

In [ ]:

model.eval()

In [ ]:
import torch
import torchvision
import torchvision.transforms as T
from PIL import Image
from transformers import GPT2Tokenizer
import numpy as np

from torch import nn as nn
import torch.nn.functional as F
def generate(image, context, cur_model, batch, ntok=20):
    for _ in range(ntok):
        out,_ = cur_model({'image': image,'input_ids': context, "index_eoq": batch["index_eoq"],
        "targets": batch["targets"],"label": batch["label"]})
        logits = out[:, -1, :]
        indices_to_remove = logits < torch.topk(logits, 10)[0][..., -1, None]
        logits[indices_to_remove] = np.NINF
        #next_tok1 = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1).squeeze(1)
        #print(next_tok1.shape)
        softmax_out = F.softmax(logits, dim=-1)
        #print(softmax_out.shape)
        next_tok = torch.argmax(softmax_out,dim=-1,keepdim=False)
        #print(next_tok.shape)
        context = torch.cat([context, next_tok.unsqueeze(-1)], dim=-1)
    return context


tokenizer = datamodule.train_dataset.tokenizer

In [ ]:
val_dataloader = datamodule.val_dataloader()

In [ ]:
len(val_dataloader)

In [ ]:
val_loader_iter = iter(val_dataloader)

In [ ]:
batch = next(val_loader_iter)

In [ ]:
T.ToPILImage()(batch["image"][0])

In [ ]:
batch = next(val_loader_iter)
context   = torch.tensor([tokenizer.encode("<|endoftext|> <image> question: "+batch["question"][0] + ' <EOQ>'+ ' answer:')]) 
out = generate(batch["image"], context, model, batch, ntok=20)
print("Question : ", batch["question"][0])
print("Model's answer : ",tokenizer.decode(out[0]).split('answer:')[1].split('<EOC>')[0])
print("Correct Answer: " + batch["answer"][0])

In [ ]:
tokenizer.decode(out[0])

In [ ]:
datamodule.train_dataset.tokenizer.all_special_ids[datamodule.train_dataset.tokenizer.all_special_tokens.index('<image>')]

In [ ]:
from efficientnet_pytorch import EfficientNet
image_encoder = EfficientNet.from_name('efficientnet-b0')

In [ ]:
image_encoder

In [ ]:
image_encoder._fc = nn.Identity()

In [ ]:
import torch
from torch import nn 

In [ ]:
new_model = nn.Sequential(*list(image_encoder.children())[:-2])

In [ ]:
image_encoder(torch.randn((1,3,224, 224))).shape

In [ ]:
# Tokenizer BERT
from transformers import AutoTokenizer
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
gpt_tokenizer = AutoTokenizer.from_pretrained("gpt2")
gpt_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [ ]:
bert_tokenizer

In [ ]:
gpt_tokenizer

In [ ]:
bert_tokenizer.name_or_path

In [ ]:
import sys 
import torch
sys.path.append('..')

import os
import pytorch_lightning as pl
from src.datasets.vqa_rad_dataset import VQRadDataModule
from src.models.multimodal.flamingo_module import FlamingoModule

from pytorch_lightning import Trainer, seed_everything
import torchvision.transforms as T

from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning import loggers as pl_loggers

from src.utils.utils import load_config
import wandb


args = load_config('/u/home/koksal/mlmi-vqa/configs','config.yaml')
    
#img_mean = (0.48,0.48,0.48)
#img_std = (0.265,0.265,0.265)
#img_mean = (0.48,0.48,0.48)
#img_std = (0.265,0.265,0.265)

augmentations = {'train':
    T.Compose(
    [   
        T.Resize((args['train']['augmentation']['resize_size'])),
        T.RandomResizedCrop(224,scale=args['train']['augmentation']['random_resize_scale'],
                            ratio=args['train']['augmentation']['random_resize_ratio']),
        T.RandomRotation((args['train']['augmentation']['random_rotation'])),
        T.ColorJitter(brightness=args['train']['augmentation']['color_jitter']['brightness'],
                        contrast=args['train']['augmentation']['color_jitter']['contrast'],
                        saturation=args['train']['augmentation']['color_jitter']['saturation'],
                        hue=args['train']['augmentation']['color_jitter']['hue']),
        T.ToTensor(),
        T.Normalize(mean=args['dataset']['mean'], std=args['dataset']['std']),
    ]),
    'val':
    T.Compose(
    [
        T.Resize((args['test']['augmentation']['resize_size'])),
        T.RandomRotation((args['test']['augmentation']['random_rotation'])),
        T.ToTensor(),
        T.Normalize(mean=args['dataset']['mean'], std=args['dataset']['std'])
    ]),
    'test':
    T.Compose(
    [
        T.Resize((args['test']['augmentation']['resize_size'])),
        T.ToTensor(),
        T.Normalize(mean=args['dataset']['mean'], std=args['dataset']['std'])
    ])
}

wandb.init(project="flamingo-research", config=args)
mimic_datamodule = VQRadDataModule(args, augmentations= augmentations)
# Data Loaders
train_loader = mimic_datamodule.train_dataloader()
val_loader = mimic_datamodule.val_dataloader()

model = FlamingoModule(args) 
wandb.watch(model, log_freq=100)

if args['pretrained']:
    print("Pretrained Flamingo Model is loaded from checkpoint : ",args['pretrained'])
    model.load_state_dict(torch.load(args['pretrained'])["state_dict"])


from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping


lr_monitor = LearningRateMonitor(logging_interval='step')


if args['model']['classification_mode']:
    checkpoint_callback = ModelCheckpoint(
                filename='{epoch}-{val_acc_epoch:.2f}-{val_total_loss_epoch:.2f}-{val_loss_generation_epoch:.2f}-{val_classification_loss_epoch:.2f}',
                monitor= 'val_acc_epoch',
                save_top_k = 3,
                save_last=True,
                mode="max")

    #early_stopping_callback = EarlyStopping(monitor="val_acc_epoch", mode="max",patience=10)
    #early_stopping_callback = EarlyStopping(monitor="val_total_loss_epoch", mode="min",patience=5)
else:
    checkpoint_callback = ModelCheckpoint(
            filename='{epoch}-{val_loss_generation_epoch:.2f}',
            monitor= 'val_loss_generation_epoch',
            save_top_k = 3, 
            save_last=True,
            mode="min")
    #early_stopping_callback = EarlyStopping(monitor="val_loss_generation_epoch", mode="min",patience=10)

#early_stopping_callback = EarlyStopping(monitor="val_acc_epoch", mode="max",patience=10)

# All our models are trained using the AdamW optimizer with global norm clipping of 1
print(args['train']['devices'])
trainer = pl.Trainer(max_epochs=args['train']['num_epochs'],
                    accelerator=args['train']['accelerator'], devices=args['train']['devices'],
                    callbacks=[lr_monitor, checkpoint_callback, ], #early_stopping_callback],
                    gradient_clip_val=args['optimizer']['gradient_clip'])

In [ ]:
batch = next(iter(train_loader))

In [ ]:
ex_layer = model.flamingo_palm.bert_decoder.bert.encoder.layer[0]

In [ ]:
model(batch)

In [11]:
from transformers import (BertGenerationDecoder, BertLayer, BertConfig)
bert_decoder = BertGenerationDecoder.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext", add_cross_attention=False, is_decoder=True, bos_token_id=None, eos_token_id=None)

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

You are using a model of type bert to instantiate a model of type bert-generation. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertGenerationDecoder: ['bert.embeddings.token_type_embeddings.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'bert.pooler.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertGenerationDecoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertGenerationDecoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassif

In [12]:
bert_decoder

BertGenerationDecoder(
  (bert): BertGenerationEncoder(
    (embeddings): BertGenerationEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertGenerationLayer(
          (attention): BertGenerationAttention(
            (self): BertGenerationSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertGenerationSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [ ]:
from src.models.multimodal.flamingo_model import Residual

In [ ]:
bertlayer = bert_decoder.base_model.encoder.layer[1]
example_layer = Residual(bertlayer)
example_layer

In [ ]:
bertlayer()

In [ ]:
bertlayer(torch.randn(2,64,768),ret)

In [ ]:
len(bertlayer(torch.randn(2,64,768))[1])

In [ ]:
bertlayer(torch.randn(2,64,768))[1][0].shape

In [ ]:
bertlayer(torch.randn(2,64,768))[1][1].shape

In [ ]:
bert_decoder = BertGenerationDecoder.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', add_cross_attention=False, is_decoder=True, bos_token_id=None, eos_token_id=None)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')

In [ ]:
tokenizer.name_or_path

In [ ]:
tokenizer.encode_plus('New york is the best liver', padding='max_length', truncation=True, max_length=30, return_tensors="pt")

In [1]:
import timm
image_encoder = timm.create_model(' ', pretrained=True)

In [2]:
image_encoder

EfficientNet(
  (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNormAct2d(
    48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
        (bn1): BatchNormAct2d(
          48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
          24, ep

In [6]:
from timm.data import resolve_data_config
config = resolve_data_config({}, model=image_encoder)
from timm.data.transforms_factory import create_transform

In [7]:
config

{'input_size': (3, 456, 456),
 'interpolation': 'bicubic',
 'mean': (0.485, 0.456, 0.406),
 'std': (0.229, 0.224, 0.225),
 'crop_pct': 0.934}

In [8]:
transforms = create_transform(**config)

In [9]:
transforms

Compose(
    Resize(size=488, interpolation=bicubic, max_size=None, antialias=None)
    CenterCrop(size=(456, 456))
    ToTensor()
    Normalize(mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]))
)